In [65]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer  # Stemmer específico para português
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
from nltk.stem import RSLPStemmer
import string


In [66]:
df = pd.read_excel('df_reviews.xlsx')

In [67]:
df.shape

(110290, 8)

In [68]:
# Função para analisar o sentimento do comentário com base na nota
def add_sentiment(df, score_column):
    """
    Adiciona uma coluna de sentimento com base na nota do comentário.
    
    Args:
        df: DataFrame contendo os comentários
        score_column: Nome da coluna contendo a nota (1-5)
    
    Returns:
        DataFrame com a coluna de sentimento adicionada
    """
    df['sentimento'] = df['score'].apply(lambda x: 
        'Positivo' if x >= 4 else 
        'Neutro' if x == 3 else 
        'Negativo')
    return df

In [69]:
df = add_sentiment(df, 'score')

In [70]:
df.head()

,user,review,score,resposta,data,app_name,store,banco,sentimento
0,Um usuário do Google,"Nao consigo abrir a conta, sempre dá erro.",2,NaN,2025-07-08 12:05:21,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo
1,Um usuário do Google,"Não consigo acessar minha conta no aplicativo,...",1,NaN,2025-07-08 08:29:24,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo
2,Um usuário do Google,Um dos melhores app financeiros....,5,NaN,2025-07-08 08:11:02,br.com.bradesco.netempresa,Google Play,Bradesco,Positivo
3,Um usuário do Google,"não é intuitivo, estou tentando excluir essa c...",1,NaN,2025-07-07 16:13:28,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo
4,Um usuário do Google,estou a quatro dias tentando recuperar minha s...,2,NaN,2025-07-07 15:16:32,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo


In [71]:
# Baixar recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')  # Para o stemmer em português

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [72]:
# 2. Função de categorização inicial por palavras-chave
def categorize_review_contextual(df, text_column):
    keywords_dict = {
        'Problemas de Login': [
            'login', 'logar', 'senha', 'senhas', 'acesso', 'acessar', 'acessando', 'acessou',
            'entrar', 'entrou', 'entrada', 'chave', 'chaves', 'token', 'bloqueado', 'bloqueada',
            'bloqueou', 'bloqueia', 'desbloqueado', 'desbloquear', 'desbloqueia'
        ],
        'Problemas Técnicos': [
            'erro', 'erros', 'bug', 'bugs', 'falha', 'falhas', 'instável', 'instaveis',
            'instabilidade', 'instabilidades', 'trava', 'travando', 'travou', 'travamento',
            'crash', 'crashou', 'crasha', 'congelado', 'congelou', 'lento', 'lenta', 'lentidão',
            'demora', 'demorado', 'fora do ar', 'fora', 'não abre', 'nao abre', 'abrir', 'abriu'
        ],
        'Usabilidade': [
            'fácil', 'facil', 'fáceis', 'dificuldade', 'difícil', 'dificil', 'complicado',
            'complicada', 'complicados', 'intuitivo', 'intuitiva', 'prático', 'pratica', 'práticos',
            'navegação', 'navegar', 'interface', 'confuso', 'confusa', 'experiência ruim', 'experiência boa'
        ],
        'Funcionalidades': [
            'função', 'funções', 'extrato', 'transferência', 'transferencias', 'transferir',
            'pagamento', 'pagamentos', 'pagar', 'boleto', 'boletos', 'consulta', 'consultar',
            'saldo', 'pix', 'cartão', 'cartões', 'investimento', 'investimentos', 'qr code', 'qrcode', 'qr'
        ],
        'Avaliação Geral': [
            'ótimo', 'ótima', 'otimo', 'otima', 'bom', 'boa', 'excelente', 'excelência', 'ruim',
            'pior', 'péssimo', 'péssima', 'horrível', 'horrivel', 'adoro', 'adorei', 'gostei',
            'recomendo', 'recomenda', 'amei', 'amei o app', 'melhor'
        ],
        'Atualizações': [
            'atualiza', 'atualizei', 'atualizado', 'atualização', 'versão', 'versões', 'nova versão',
            'nova', 'novo', 'mudança', 'mudanças', 'melhoria', 'melhorias', 'antiga', 'antigo',
            'layout novo', 'interface nova'
        ]
    }

    def classificar_texto(texto):
        texto = str(texto).lower()
        for categoria, palavras in keywords_dict.items():
            if any(palavra in texto for palavra in palavras):
                return categoria
        return 'Outros'

    df['categoria_inicial'] = df[text_column].apply(classificar_texto)
    return df

In [75]:
df = categorize_review_contextual(df, 'review')

In [76]:
df.head()

,user,review,score,resposta,data,app_name,store,banco,sentimento,categoria_inicial
0,Um usuário do Google,"Nao consigo abrir a conta, sempre dá erro.",2,NaN,2025-07-08 12:05:21,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas Técnicos
1,Um usuário do Google,"Não consigo acessar minha conta no aplicativo,...",1,NaN,2025-07-08 08:29:24,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login
2,Um usuário do Google,Um dos melhores app financeiros....,5,NaN,2025-07-08 08:11:02,br.com.bradesco.netempresa,Google Play,Bradesco,Positivo,Avaliação Geral
3,Um usuário do Google,"não é intuitivo, estou tentando excluir essa c...",1,NaN,2025-07-07 16:13:28,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login
4,Um usuário do Google,estou a quatro dias tentando recuperar minha s...,2,NaN,2025-07-07 15:16:32,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login


In [77]:
# 3. Treinar modelo supervisionado com base nas categorias identificadas (exceto "Outros")
df_modelo = df[df['categoria_inicial'] != 'Outros'].copy()

In [78]:
X = df_modelo['review']
y = df_modelo['categoria_inicial']

In [79]:
# Lista básica de stopwords (caso não consiga usar nltk)
stopwords_pt = [
    'a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com',
    'como', 'da', 'das', 'de', 'dela', 'dele', 'deles', 'demais', 'depois', 'do', 'dos', 'e', 'ela',
    'elas', 'ele', 'eles', 'em', 'entre', 'era', 'essa', 'essas', 'esse', 'esses', 'esta', 'está',
    'estamos', 'estas', 'estava', 'este', 'estes', 'estou', 'eu', 'foi', 'foram', 'fui', 'há', 'isso',
    'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'na',
    'nas', 'não', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'o', 'os', 'ou',
    'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'se', 'seja',
    'sem', 'seu', 'seus', 'só', 'sua', 'suas', 'também', 'te', 'tem', 'tenho', 'tendo', 'ter', 'teu',
    'teus', 'tive', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês'
]

In [80]:
# Divide e treina modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

1. TfidfVectorizer(stop_words=stopwords_pt)
Converte o texto bruto (reviews) em uma matriz numérica esparsa com os pesos TF-IDF.

TF-IDF (Term Frequency–Inverse Document Frequency) mede a importância de cada palavra no contexto de todos os documentos.

A opção stop_words=stopwords_pt remove palavras comuns do português (como "de", "o", "e", etc.) que não carregam informação relevante.

Exemplo de entrada: "não consigo acessar minha conta"

Saída: Vetor numérico com os pesos TF-IDF de cada termo relevante.

2. LogisticRegression(max_iter=1000, random_state=42)
Classificador que aprende a prever uma categoria com base nos vetores TF-IDF.

max_iter=1000 garante que o modelo tenha iterações suficientes para convergir.

random_state=42 assegura reprodutibilidade dos resultados.

💡 Objetivo do pipeline:
Ele automatiza o fluxo:

Texto cru ⟶ TF-IDF ⟶ Classificação (ex: Problemas Técnicos, Usabilidade...)

In [81]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords_pt)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

In [82]:

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'dele', 'deles', 'demais',
                                             'depois', 'do', 'dos', 'e', 'ela',
                                             'elas', 'ele', 'eles', 'em',
                                             'entre', ...])),
                ('clf', LogisticRegression(max_iter=1000, random_state=42))])

In [83]:
# Avaliação opcional
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

                    precision    recall  f1-score   support

      Atualizações       0.88      0.71      0.79       244
   Avaliação Geral       0.98      1.00      0.99      9436
   Funcionalidades       0.89      0.94      0.91      1135
Problemas Técnicos       0.95      0.94      0.94      2205
Problemas de Login       1.00      0.95      0.97      1879
       Usabilidade       0.99      0.96      0.97      2398

          accuracy                           0.97     17297
         macro avg       0.95      0.91      0.93     17297
      weighted avg       0.97      0.97      0.97     17297



In [84]:
# Filtra apenas os reviews classificados como 'Outros' e que têm texto válido
df_outros = df[df['categoria_inicial'] == 'Outros'].copy()
df_outros_validos = df_outros[df_outros['review'].notna()].copy()


In [85]:
# Faz a previsão com o modelo treinado
df_outros_validos['categoria_prevista'] = pipeline.predict(df_outros_validos['review'])


In [86]:
# Preenche o restante com NaN para manter estrutura
df_outros['categoria_prevista'] = df_outros_validos['categoria_prevista']

In [87]:
# Junta os reviews previamente classificados com os reclassificados
df_rotulados = df[df['categoria_inicial'] != 'Outros'].copy()
df_rotulados['categoria_prevista'] = df_rotulados['categoria_inicial']


In [88]:

# Resultado final unificado
df_final = pd.concat([df_rotulados, df_outros], ignore_index=True)

In [89]:
df_final

,user,review,score,resposta,data,app_name,store,banco,sentimento,categoria_inicial,categoria_prevista
0,Um usuário do Google,"Nao consigo abrir a conta, sempre dá erro.",2,NaN,2025-07-08 12:05:21,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas Técnicos,Problemas Técnicos
1,Um usuário do Google,"Não consigo acessar minha conta no aplicativo,...",1,NaN,2025-07-08 08:29:24,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login,Problemas de Login
2,Um usuário do Google,Um dos melhores app financeiros....,5,NaN,2025-07-08 08:11:02,br.com.bradesco.netempresa,Google Play,Bradesco,Positivo,Avaliação Geral,Avaliação Geral
3,Um usuário do Google,"não é intuitivo, estou tentando excluir essa c...",1,NaN,2025-07-07 16:13:28,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login,Problemas de Login
4,Um usuário do Google,estou a quatro dias tentando recuperar minha s...,2,NaN,2025-07-07 15:16:32,br.com.bradesco.netempresa,Google Play,Bradesco,Negativo,Problemas de Login,Problemas de Login
...,...,...,...,...,...,...,...,...,...,...,...
110285,Um usuário do Google,Da muitos problemas...,2,NaN,2018-10-22 17:41:23,br.livetouch.safra.empresas,Google Play,Safra,Negativo,Outros,Problemas Técnicos
110286,Um usuário do Google,odiando,1,NaN,2018-10-09 17:04:17,br.livetouch.safra.empresas,Google Play,Safra,Negativo,Outros,Avaliação Geral
110287,Um usuário do Google,App não funciona quando se está conectado ao 4...,1,NaN,2018-09-24 12:24:56,br.livetouch.safra.empresas,Google Play,Safra,Negativo,Outros,Avaliação Geral
110288,Um usuário do Google,App não faz Ted uma vergonha,1,NaN,2018-09-14 15:39:15,br.livetouch.safra.empresas,Google Play,Safra,Negativo,Outros,Avaliação Geral


In [ ]:
df_final.head(100)

In [91]:
  df_final.to_csv('df_final.csv', index=False, encoding='utf-8-sig')

In [90]:
df_final.to_excel('df_final.xlsx', index=False)


In [ ]:
# Função para limpar e pré-processar o texto
def preprocess_text(text):
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        # Remover pontuação
        #    cria um mapeamento que apaga todos os caracteres de string.punctuation
        translator = str.maketrans('', '', string.punctuation)
        text = text.translate(translator)

        # Remover caracteres especiais e números
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Tokenização
        tokens = nltk.word_tokenize(text)
        # Remover stopwords em português
        stop_words = set(stopwords.words('portuguese'))
        tokens = [word for word in tokens if word not in stop_words]
        # Stemming (redução das palavras ao seu radical)
        stemmer = RSLPStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        # Reconstruir texto
        return ' '.join(tokens)
    else:
        return ''

In [ ]:
# Função para categorizar comentários usando regras baseadas em palavras-chave
def categorize_by_keywords(df, text_column):
    """
    Categoriza comentários com base em dicionários de palavras-chave.
    
    Args:
        df: DataFrame contendo os comentários
        text_column: Nome da coluna contendo o texto
    
    Returns:
        DataFrame com a coluna de categoria adicionada
    """
    # Definir categorias e palavras-chave relevantes para app bancário
    keywords_dict = {
        'Problemas de Login': ['login', 'senha', 'acesso', 'entrar', 'chave', 'seguranca', 'token', 'bloquead'],
        'Problemas Técnicos': ['erro', 'fecha', 'bug', 'trava', 'cai', 'instavel', 'lento', 'carrega', 'funcionou'],
        'Usabilidade': ['facil', 'dificil', 'complicad', 'simples', 'intuitivo', 'pratico', 'rapido', 'interface'],
        'Funcionalidades': ['funcao', 'extrato', 'transfer', 'pagamento', 'boleto', 'consulta', 'saldo', 'pix'],
        'Avaliação Geral': ['otimo', 'bom', 'excelente', 'ruim', 'pessimo',  'péssimo', 'horrivel', 'melhor', 'pior', 'recomend'],
        'Atualizações': ['atualiza', 'versao', 'nova', 'antigo', 'mudanca', 'melhora']
    }
    
    # Cria uma cópia do DataFrame para não modificar o original
    result_df = df.copy()
    
    # Adiciona uma coluna para a categoria
    result_df['categoria'] = 'Outros'
    
    # Itera sobre cada linha do DataFrame
    for idx, row in result_df.iterrows():
        text = str(row[text_column]).lower()
        
        # Verifica cada categoria
        for category, keywords in keywords_dict.items():
            # Se qualquer palavra-chave estiver presente no texto, atribui a categoria
            if any(keyword in text for keyword in keywords):
                result_df.at[idx, 'categoria'] = category
                break
                
    return result_df

In [ ]:
# Função para analisar o sentimento do comentário com base na nota
def add_sentiment(df, score_column):
    """
    Adiciona uma coluna de sentimento com base na nota do comentário.
    
    Args:
        df: DataFrame contendo os comentários
        score_column: Nome da coluna contendo a nota (1-5)
    
    Returns:
        DataFrame com a coluna de sentimento adicionada
    """
    df['sentimento'] = df['score'].apply(lambda x: 
        'Positivo' if x >= 4 else 
        'Neutro' if x == 3 else 
        'Negativo')
    return df

In [ ]:
# Função para treinar e avaliar modelos de machine learning
def train_ml_categorization(X_train, X_test, y_train, y_test):
    """
    Treina e avalia diferentes modelos de classificação.
    
    Args:
        X_train, X_test: Features de treino e teste
        y_train, y_test: Labels de treino e teste
    
    Returns:
        O melhor modelo treinado
    """
    # Define modelos para testar
    models = {
        'Naive Bayes': MultinomialNB(),
        'Regressão Logística': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'SVM': LinearSVC()
    }
    
    best_score = 0
    best_model = None
    
    # Avalia cada modelo
    for name, model in models.items():
        print(f"\nTreinando {name}...")
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"Acurácia: {score:.4f}")
        
        y_pred = model.predict(X_test)
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred))
        
        # Salva o melhor modelo
        if score > best_score:
            best_score = score
            best_model = model
    
    return best_model

In [ ]:
# Função para visualizar a distribuição das categorias
def plot_categories_distribution(df, category_column):
    plt.figure(figsize=(12, 6))
    sns.countplot(y=df[category_column], order=df[category_column].value_counts().index)
    plt.title('Distribuição das Categorias')
    plt.xlabel('Número de Comentários')
    plt.ylabel('Categoria')
    plt.tight_layout()
    plt.savefig('categorias_distribuicao.png')
    plt.close()

In [ ]:
# Função para visualizar cruzamento entre categoria e sentimento
def plot_category_sentiment(df, category_column, sentiment_column):
    plt.figure(figsize=(14, 8))
    ct = pd.crosstab(df[category_column], df[sentiment_column])
    ct_percent = ct.div(ct.sum(axis=1), axis=0)
    ct_percent.plot(kind='barh', stacked=True, colormap='viridis')
    plt.title('Distribuição de Sentimento por Categoria')
    plt.xlabel('Proporção')
    plt.ylabel('Categoria')
    plt.legend(title='Sentimento')
    plt.tight_layout()
    plt.savefig('categoria_sentimento.png')
    plt.close()

In [ ]:
# limpa valores ausentes
# df = df.dropna()

In [ ]:
# df.shape

In [ ]:
# Função principal para processar o dataset
def main():
    # Carrega os dados
    # Ajuste o separador conforme necessário (parece ser um CSV ou TSV)
    try:
        df = pd.read_excel('df_reviews.xlsx')
        # df = df.dropna()
    except:
        # Tente outro separador se o primeiro não funcionar
        df = pd.read_excel('df_reviews.xlsx')
        # df = df.dropna()

    # Verifica as colunas disponíveis
    print("Colunas disponíveis no dataset:")
    print(df.columns.tolist())
    
    # Mostra as primeiras linhas do dataset
    print("\nPrimeiras linhas do dataset:")
    print(df.head())
    
    # Informações básicas
    print("\nInformações básicas sobre o dataset:")
    print(f"Número de comentários: {len(df)}")
    print(f"Distribuição das notas:")
    print(df['score'].value_counts().sort_index())
    
    # Pré-processamento dos comentários
    print("\nRealizando pré-processamento dos comentários...")
    df['comentario_processado'] = df['review'].apply(preprocess_text)
    
    # Adiciona sentimento com base na nota
    df = add_sentiment(df, 'score')
    
    # ABORDAGEM 1: Categorização baseada em palavras-chave
    print("\n==== ABORDAGEM 1: Categorização por palavras-chave ====")
    categorized_df = categorize_by_keywords(df, 'comentario_processado')
    
    print("Distribuição de categorias:")
    category_counts = categorized_df['categoria'].value_counts()
    print(category_counts)
    
    # Visualiza a distribuição das categorias
    plot_categories_distribution(categorized_df, 'categoria')
    
    # Visualiza relação entre categoria e sentimento
    plot_category_sentiment(categorized_df, 'categoria', 'sentimento')
    
    # ABORDAGEM 2: Categorização por machine learning
    print("\n==== ABORDAGEM 2: Categorização por machine learning ====")
    
    # Usar as categorias da abordagem por palavras-chave como rótulos para treinar o modelo
    X = categorized_df['comentario_processado']
    y = categorized_df['categoria']
    
    # Divide em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Vetorização do texto usando TF-IDF
    print("\nVetorizando os textos...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Treina e avalia os modelos
    best_model = train_ml_categorization(X_train_vec, X_test_vec, y_train, y_test)
    
    # Cria um pipeline com o melhor modelo
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', best_model)
    ])
    
    # Treina o pipeline com todos os dados
    pipeline.fit(X, y)
       
    # Salva o modelo treinado
    dump(pipeline, 'modelo_categorizacao_app_bradesco.joblib')
    print("Modelo salvo como 'modelo_categorizacao_app_bradesco.joblib'")
    
    # Salva o DataFrame categorizado
    categorized_df.to_csv('comentarios_categorizados.csv', index=False, encoding='utf-8-sig')
    categorized_df.to_excel('comentarios_categorizados.xlsx', index=False)
    
    print("Dataset categorizado salvo como 'comentarios_categorizados.csv'")


if __name__ == "__main__":
    main()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
df.info()

In [ ]:

    # Exemplo de aplicação: categorizar novos comentários
    print("\n==== APLICAÇÃO: Categorizar novos comentários ====")
    
    # Exemplos de novos comentários (ajuste conforme necessário)
    novos_comentarios = [
        "Não consigo fazer login, pede a chave de segurança mas não reconhece",
        "App muito bom, fácil de usar e com todas as funções que preciso",
        "Fiz a atualização e agora fica travando quando tento fazer transferência",
        "Não consigo ver meu extrato, dá erro toda vez que tento acessar",
        "Excelente aplicativo, muito prático para gerenciar minha empresa"
    ]
    
    # Pré-processa os novos comentários
    novos_comentarios_proc = [preprocess_text(texto) for texto in novos_comentarios]
    
    # Prevê as categorias
    categorias_previstas = pipeline.predict(novos_comentarios_proc)
    
    # Exibe os resultados
    print("\nResultados da categorização:")
    for comentario, categoria in zip(novos_comentarios, categorias_previstas):
        print(f"Comentário: {comentario}")
        print(f"Categoria prevista: {categoria}\n")

In [ ]:

    # Dicas de insights que podem ser extraídos
    print("\n==== INSIGHTS POTENCIAIS ====")
    print("1. Categorias mais comuns de problemas reportados")
    print("2. Relação entre categorias e sentimento (notas)")
    print("3. Evolução temporal das categorias (análise de tendência)")
    print("4. Comparação entre versões do aplicativo")
    print("5. Palavras mais frequentes em cada categoria")